In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import (get_training_observations, 
                     get_training_labels)
import pandas as pd
import numpy as np
from joblib import dump
from joblib import load

# Model specific imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import metrics
tf.get_logger().setLevel('INFO')

import optuna

In [2]:
# Set up data
x_train = get_training_observations()
y_train = get_training_labels()
le = load('../model_joblibs/final_labelEncoder.joblib')
    
y_train_transformed = le.transform(y_train['EMPO_3'])
y_train_transformed

x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train_transformed, test_size=0.2, random_state=0, stratify=y_train_transformed)   


Getting all training observations from 'metagenome_classification.db'...
Getting all training labels from 'metagenome_classification.db'...


In [3]:
K = tf.keras.backend

def build_model(
    activation='relu',
    hidden_layer_sizes=[],
    final_layer_activation='softmax',
    dropout=0.0,
    optimizer='Adam',
    learning_rate=0.01,
    kernel_regularizer=1e-5,
    bias_regularizer=1e-5,
    activity_regularizer=1e-5,
    n_classes=15
):
    """Build a multi-class logistic regression model using Keras.

    Args:
      n_classes: Number of output classes in the dataset.
      hidden_layer_sizes: A list with the number of units in each hidden layer.
      activation: The activation function to use for the hidden layers.
      optimizer: The optimizer to use (SGD, Adam).
      learning_rate: The desired learning rate for the optimizer.

    Returns:
      model: A tf.keras model (graph).
    """
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    
    for hidden_layer_size in hidden_layer_sizes:
        if hidden_layer_size > n_classes:
            model.add(tf.keras.layers.Dense(
                hidden_layer_size,
                activation=activation,
                kernel_regularizer=tf.keras.regularizers.L2(kernel_regularizer),
                bias_regularizer=tf.keras.regularizers.L2(bias_regularizer),
                activity_regularizer=tf.keras.regularizers.L2(activity_regularizer)
            ))
            if dropout > 0:
                model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Dense(
        n_classes,
        activation=final_layer_activation,
        kernel_regularizer=tf.keras.regularizers.L2(kernel_regularizer),
        bias_regularizer=tf.keras.regularizers.L2(bias_regularizer),
        activity_regularizer=tf.keras.regularizers.L2(activity_regularizer)
    ))
    opt = None
    if optimizer == "SGD":
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "Adam":
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        raise f"Unsupported optimizer, {optimizer}"
    model.compile(
        loss="categorical_crossentropy", optimizer=opt, 
        metrics=[tfa.metrics.F1Score(num_classes=15, average='weighted')]
    )
    return model
    

In [4]:
def objective_3_layer(trial, x_tr=x_tr, x_val=x_val, y_tr=y_tr, y_val=y_val):
    
    # Set up XGBoost hyperparameter space
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01),
        'hidden_layer_size1': trial.suggest_int('hidden_layer_size1', 0, 1024),
        'hidden_layer_size2': trial.suggest_int('hidden_layer_size2', 0, 1024),
        'hidden_layer_size3': trial.suggest_int('hidden_layer_size3', 0, 1024),
        'kernel_regularizer': trial.suggest_float('kernel_regularizer', 1e-10, 1e-4),
        'bias_regularizer': trial.suggest_float('bias_regularizer', 1e-10, 1e-4),
        'activity_regularizer': trial.suggest_float('activity_regularizer', 1e-10, 1e-4),
        'dropout': trial.suggest_float('dropout', 0, 0.1)
    }
    
    hidden_layer_sizes = [params.pop('hidden_layer_size1'), params.pop('hidden_layer_size2'), params.pop('hidden_layer_size3')]
    
    y_tr_one_hot = K.one_hot(K.cast(y_tr, dtype='int64'), 15)
    y_val_one_hot = K.one_hot(K.cast(y_val, dtype='int64'), 15)
    
    # Create model
    nn = build_model(
        **params,
        n_classes=15,
        optimizer='Adam',
        final_layer_activation='softmax',
        hidden_layer_sizes=hidden_layer_sizes
    )
    
    # Fit model
    nn.fit(x=x_tr,
           y=y_tr_one_hot,
           epochs=50,
           batch_size=64,
           verbose=0)
    
    evaluation = nn.evaluate(x=x_val, y=y_val_one_hot, verbose=0, return_dict=True)
    f1_score = evaluation['f1_score']
    loss = evaluation['loss']
    
    print(f'F1 Score: {f1_score}; Loss={loss}')
    
    return f1_score
    

In [ ]:
study_nn_3 = optuna.create_study(direction='maximize')
study_nn_3.optimize(objective_3_layer, n_trials=100)

print('Number of finished trials:', len(study_nn_3.trials))
print('Best trial:', study_nn_3.best_trial.params)

[I 2022-08-03 18:06:57,360] A new study created in memory with name: no-name-577c86e0-718f-4470-a57f-58e3b8375092


Metal device set to: Apple M1 Pro


2022-08-03 18:06:57.362218: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-03 18:06:57.362408: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-08-03 18:06:57.452861: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-03 18:06:58.010425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 18:07:12.758035: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 18:07:12,973] Trial 0 finished with value:

F1 Score: 0.898534893989563; Loss=0.5751761198043823


2022-08-03 18:07:13.455545: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 18:07:24.583407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 18:07:24,791] Trial 1 finished with value: 0.8586052656173706 and parameters: {'learning_rate': 0.000979931377394091, 'hidden_layer_size1': 406, 'hidden_layer_size2': 449, 'hidden_layer_size3': 1014, 'kernel_regularizer': 5.655522744433374e-05, 'bias_regularizer': 8.583939977149671e-05, 'activity_regularizer': 4.5311083119561646e-06, 'dropout': 0.005701038908311507}. Best is trial 0 with value: 0.898534893989563.


...

2022-08-03 18:16:13.936438: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 18:16:29.176719: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 18:16:29,790] Trial 38 finished with value: 0.9148057699203491 and parameters: {'learning_rate': 0.006694821121334568, 'hidden_layer_size1': 268, 'hidden_layer_size2': 832, 'hidden_layer_size3': 693, 'kernel_regularizer': 1.4051913157721362e-06, 'bias_regularizer': 8.9631021419747e-05, 'activity_regularizer': 3.3745400311365155e-05, 'dropout': 0.09932760743645741}. Best is trial 28 with value: 0.9307509660720825.


F1 Score: 0.9148057699203491; Loss=0.5683242678642273


2022-08-03 18:16:30.480777: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 18:16:46.049433: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 18:16:46,760] Trial 39 finished with value: 0.9114333391189575 and parameters: {'learning_rate': 0.006761924111537636, 'hidden_layer_size1': 283, 'hidden_layer_size2': 837, 'hidden_layer_size3': 710, 'kernel_regularizer': 2.253856468529385e-05, 'bias_regularizer': 9.75838114286779e-05, 'activity_regularizer': 3.2423186141091186e-05, 'dropout': 0.09833556757251949}. Best is trial 28 with value: 0.9307509660720825.


F1 Score: 0.9114333391189575; Loss=0.533988356590271


2022-08-03 18:16:47.284772: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


* Trial 29 finished with value: 0.9176479578018188 and parameters: {'learning_rate': 0.006120059584156001, 'hidden_layer_size1': 6, 'hidden_layer_size2': 871, 'hidden_layer_size3': 424, 'kernel_regularizer': 7.04892717757341e-07, 'bias_regularizer': 8.077172989474054e-05, 'activity_regularizer': 4.125772221139609e-05, 'dropout': 0.03581861197995262}. Best is trial 29 with value: 0.9176479578018188.

* **With stratified sampling:** Trial 39 finished with value: 0.9002330303192139 and parameters: {'learning_rate': 0.007284468192554461, 'hidden_layer_size1': 399, 'hidden_layer_size2': 928, 'hidden_layer_size3': 208, 'kernel_regularizer': 6.137914541289313e-06, 'bias_regularizer': 9.963601430040577e-05, 'activity_regularizer': 7.078406366492952e-05, 'dropout': 0.02909313212948919}. Best is trial 39 with value: 0.9002330303192139

* **With stratified sampling:** Trial 28 finished with value: 0.9307509660720825 and parameters: {'learning_rate': 0.00399057337146822, 'hidden_layer_size1': 321, 'hidden_layer_size2': 571, 'hidden_layer_size3': 233, 'kernel_regularizer': 2.412083737943694e-07, 'bias_regularizer': 3.7539762303542793e-05, 'activity_regularizer': 4.493669155490493e-05, 'dropout': 0.0816250430781725}. Best is trial 28 with value: 0.9307509660720825.